# Exercise 1: Lightcurve analysis

The exercise will be gradeded with between one and three stars: One star for completing the notebook, two stars for doing so systematically and with well motivated answers and three starsfor comprehensive and accurate uncertainty estimates. 

# Optical data analysis with Lightkurve

Lightkurve is a python library that offers a user-friendly way to analyse time series data on the brightness of planets, stars, and galaxies. The package is focused on supporting science with NASA’s Kepler and TESS space telescopes, but can equally be used to analyze light curves obtained by your backyard telescope.

To download the package, run the following command in a terminal window:
```
pip install lightkurve
```
then close and reopen this notebook file.

## What are `TargetPixelFile` objects?

Target Pixel Files (TPFs) are a file common to Kepler/K2 and the TESS mission. They contain movies of the pixel data centered on a single target star.

TPFs can be thought of as stacks of images, with one image for every timestamp the telescope took data. Each timestamp is referred to as a **cadence**. These images are cut out 'postage stamps' of the full observation to make them easier to work with. 

TPFs are given in FITS files, which you can read more about [here](https://fits.gsfc.nasa.gov/fits_primer.html). *Lightkurve* includes tools for you to work directly with these files easily and intuitively.

To load a FITS file from a local path or remote url, simply call Lightkurve's [read](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.read.html?highlight=read#lightkurve.LightCurve.read) function using the location of the file as the parameter:

In [ ]:
import lightkurve as lk
tpf = lk.read("https://archive.stsci.edu/pub/kepler/target_pixel_files/0069/006922244/kplr006922244-2010078095331_lpd-targ.fits.gz")

You can also search for the url automatically using the [search_targetpixelfile()](https://docs.lightkurve.org/reference/api/lightkurve.search_targetpixelfile.html?highlight=search_targetpixelfile) function. This will search for the right file in the [MAST data archive](https://archive.stsci.edu/kepler/) which holds all of the Kepler and K2 data.
In this case we want the Target Pixel File with Kepler ID 6922244 for Quarter 4 (Kepler's observations were split into quarters of a year):

In [ ]:
from lightkurve import search_targetpixelfile
tpf = search_targetpixelfile('KIC 6922244', author="Kepler", quarter=4, cadence="long").download()

You can also pass the name of the target or its astronomical coordinates as a parameter to `search_targetpixelfile()`.

The above code has created a variable named `tpf` which is a Python object of type `KeplerTargetPixelFile`:

In [ ]:
tpf

We can access lots of meta data using this object in a simple way. For example, we can find the mission name, and the quarter that the data was taken in by typing the following:

In [ ]:
tpf.meta['MISSION']

In [ ]:
tpf.meta['QUARTER']

You can find the full list of properties in the [API documentation](https://docs.lightkurve.org/reference/api/lightkurve.targetpixelfile.KeplerTargetPixelFile.html#lightkurve.targetpixelfile.KeplerTargetPixelFile) on this object.

The most interesting data in a `KeplerTargetPixelFile` object are the `flux` and `time` values which give access to the brightness of the observed target over time. You can access the timestamps of the observations using the `time` property:

In [ ]:
tpf.time

By default, `time` is in the Kepler-specific *Barycentric Kepler Julian Day* format (BKJD).

Because this is an AstroPy Time object, you access to human-readable ISO timestamps using the `time.iso` property:

In [ ]:
tpf.time.iso

**Beware:** these timestamps are in the Solar System Barycentric frame (TDB) and do not include corrections for light travel time or leap seconds.  To use a different time scale, such as the Earth-centered UTC system, you can use [AstroPy's time scale conversion features](http://docs.astropy.org/en/stable/time/#time-scale).  For example: 

In [ ]:
tpf.time.utc.iso

Next, let's look at the actual image data, which is available via the `flux` property:

In [ ]:
tpf.flux.shape

The `flux` data is a 4116x5x5 array in units electrons/second. The first axis is the time axis, and the images themselves are 5 pixels by 5 pixels. You can use the `plot` method on the `KeplerTargetPixelFile` object to view the data. (By default, this will show just one cadence of the data. But you can pass the cadence you want to look at to the `frame` keyword if you would like to check a particular flux point for thruster firings, cosmic rays or asteroids.)

In [ ]:
%matplotlib inline
tpf.plot(frame=0)

The values shown in this image are also directly accessible as an array:

In [ ]:
tpf.flux[0]

You can use normal `numpy` methods on these to find the shape, mean etc!

We can now turn this Target Pixel File into a light curve, with a single flux value for every time value. Each of the pixels are 4 arcseconds across. The point spread function (PSF) of the telescope causes the light from the star fall onto several different pixels, which can be seen in the image above. Because of this spreading, we have to sum up many pixels to collect all the light from the source. To do this we sum up all the pixels in an **aperture**. An aperture is a pixel mask, where we take only the pixels related to the target. 

The *Kepler* pipeline adds an aperture mask to each target pixel file. This aperture determines which pixels are summed to create a 1-D light curve of the target. There are some science cases where you might want to create a different aperture. For example, there may be a nearby contaminant or you may want to measure the background. 

The standard pipeline aperture is easily accessed in a `KeplerTargetPixelFile` object using [tpf.pipeline_mask](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.pipeline_mask.html?highlight=pipeline_mask), which is a boolean array:

In [ ]:
tpf.pipeline_mask

We can also plot this aperture over the target pixel file above to see if the flux of the star is all contained within the aperture.

In [ ]:
tpf.plot(aperture_mask=tpf.pipeline_mask)

Now that we have the aperture we can create a Simple Aperture Photometry light curve in the next section.

Finally, note that you can inspect all the raw metadata of the target by taking a look at the 'header' of the FITS file, which contains information about the data set. Let's just print the first 10 lines:

In [ ]:
tpf.get_header()[:10]

We can look at the values in the second extension of the fits file by accessing the AstroPy FITS `HDUList` object. For example, to look at all the column titles:

In [ ]:
tpf.hdu[1].header['TTYPE*']

___
## What are `LightCurve` objects?

`LightCurve` objects are data objects which encapsulate the brightness of a star over time. They provide a series of common operations, for example folding, binning, plotting, etc. There are a range of subclasses of `LightCurve` objects specific to telescopes, including `KeplerLightCurve` for Kepler and K2 data and `TessLightCurve` for TESS data.

Although *lightkurve* was designed with Kepler, K2 and TESS in mind, these objects can be used for a range of astronomy data.

You can create a `LightCurve` object from a `TargetPixelFile` object using Simple Aperture Photometry. Aperture Photometry is the simple act of summing up the values of all the pixels in a pre-defined aperture, as a function of time. By carefully choosing the shape of the aperture mask, you can avoid nearby contaminants or improve the strength of the specific signal you are trying to measure relative to the background.

To demonstrate, let's create a `KeplerLightCurve` from a `KeplerTargetPixelFile`.

In [ ]:
# Convert the target pixel file into a light curve using the pipeline-defined aperture mask.
lc = tpf.to_lightcurve(aperture_mask=tpf.pipeline_mask)

We've built a new `KeplerLightCurve` object called `lc`. Note in this case we've passed an **aperture_mask** to the [to_lightcurve](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.to_lightcurve.html?highlight=to_lightcurve#lightkurve.KeplerTargetPixelFile.to_lightcurve) method. The default is to use the *Kepler* pipeline aperture. (You can pass your own aperture, which is a boolean `numpy` array.) By summing all the pixels in the aperture we have created a Simple Aperture Photometry (SAP) lightcurve.

`KeplerLightCurve` has many useful functions that you can use. As with Target Pixel Files you can access the meta data very simply:

In [ ]:
lc.meta['MISSION']

In [ ]:
lc.meta['QUARTER']

And you still have access to time and flux attributes. In a light curve, there is only one flux point for every time stamp:

In [ ]:
lc.time

In [ ]:
lc.flux

In [ ]:
lc.flux_err

Now we can use the built in [plot](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.plot.html?highlight=plot#lightkurve.LightCurve.plot) function on the `KeplerLightCurve` object to plot the time series. You can pass `plot` any keywords you would normally pass to `matplotlib.pyplot.plot`.

In [ ]:
%matplotlib inline
lc.plot()

There are a set of useful functions in `LightCurve` objects which you can use to work with the data. These include:

* [flatten()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.flatten.html?highlight=flatten#lightkurve.LightCurve.flatten): Remove long term trends using a [Savitzky–Golay filter](https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter)

* [remove_outliers()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.remove_outliers.html?highlight=remove_outliers#lightkurve.LightCurve.remove_outliers): Remove outliers using simple sigma clipping

* [remove_nans()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.remove_nans.html?highlight=remove_nans#lightkurve.LightCurve.remove_nans): Remove infinite or NaN values (these can occur during thruster firings)

* [fold()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.fold.html?highlight=fold#lightkurve.LightCurve.fold): Fold the data at a particular period

* [bin()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.bin.html?highlight=bin#lightkurve.LightCurve.bin): Reduce the time resolution of the array, taking the average value in each bin.

We can use these simply on a light curve object

In [ ]:
flat_lc = lc.flatten(window_length=401)
flat_lc.plot()

Note that the y-axis has changed -- the flux values have been normalised as the light curve is flattened.

You can see that there are periodic 'dips' in the lightcurve, where the flux decreases rapidly. The Kepler Mission was a NASA-directed program tasked with searching a region of the sky for terrestrial planets that transit, or cross in front of (and therefore, for a certain time, make dimmer) the stars that they orbit with respect to Earth. Looking at the flattened light curve, it is is very likely that this is a transit light curve from an exoplanet!

In order to find out more about the system, we need to deduce the transit period (i.e. the time it takes the planet to orbit the star) and duration.

<img style="float: right;" src="https://docs.astropy.org/en/stable/timeseries/bls-1.png" alt="Box Least Squares" width="600px"/>

The most common method used to identify transiting exoplanets is the Box Least Squares (BLS) periodogram analysis. BLS works by modeling a transit using an upside-down top hat with four parameters: period, duration, depth, and reference time. These can be seen in the figure to the right, from the [astropy.timeseries](https://docs.astropy.org/en/stable/timeseries/) implementation of BLS.

These parameters are then optimized by minimizing the square difference between the BLS transit model and the observation. For more information about BLS, please see the [Astropy documentation](https://docs.astropy.org/en/stable/timeseries/bls.html).

To create a `BoxLeastSquaresPeriodogram`, use the `LightCurve` method [to_periodogram](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.to_periodogram.html?highlight=to_periodogram), and pass in the string `'bls'` to specify the type of periodogram object you want to create. This method also optionally takes an array of periods (in days) to search, which we will set from 1–20 days to limit our search to short-period planets. We do so using the [numpy.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) function.

In [ ]:
import numpy as np
# Create array of periods to search
period = np.linspace(1, 20, 10000)
# Create a BLSPeriodogram
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500);
bls.plot()

The plot above shows the power, or the likelihood of the BLS fit, for each of the periods in the array we passed in. This plot shows a handful of high-power peaks at discrete periods, which is a good sign that a transit has been identified. The highest power spike shows the most likely period, while the lower power spikes are harmonics of the period (at multiples of the period).

We can determine the most likely BLS period by taking the period value at maximum power.

In [ ]:
planet_period = bls.period_at_max_power
planet_period

You can also determine the most likely BLS duration.

In [ ]:
delta_t = bls.duration_at_max_power
delta_t

To confirm that this period corresponds to a transit signal, we can phase-fold the light curve using this value and plot it.

In [ ]:
folded_lc = flat_lc.fold(period=planet_period)
folded_lc.plot()

This is quite convincing!

With these very quick methods, we have determined the period that an exoplanet orbits a star - pretty cool right?

The actual period of the planet was determined to be 3.5225 days. If we change our folding period to this, you can see that the light curve is less 'noisy' around the transit point.

In [ ]:
folded_lc = flat_lc.fold(period=3.5225)
folded_lc.plot()

Then, we can bin the light curve, which takes the average value in each bin.

In [ ]:
binned_lc = folded_lc.bin(time_bin_size=0.01)
binned_lc.plot()

Or we can do these all in a single (long) line!

In [ ]:
lc.remove_nans().flatten(window_length=401).fold(period=3.5225).bin(time_bin_size=0.01).plot()

This light curve is for Kepler-8, a 1.2 $\mathrm{M_\odot}$ star with a radius of 1.486 $\mathrm{R_\odot}$, and an example of one of Kepler's discoveries. Kepler-8 has a gas giant planet in its orbit (known as Kepler-8b), which produces the transit (dimming) in the light curve. The planet orbits the star every 3.5225 days, which is the period at which we folded the light curve.

If the host star's flux is normally $F_*$ then during the transit it may attain a minimum flux determined by the maximum fraction of the host's area covered by the planet:
$$F_{\mathrm{min}} = F_* \left[ 1 - \left( \frac{R_p}{R_*} \right)^2\right],$$
where $R_*$ is the radius of the star, $R_p$ is the radius of the planet, and assuming that the host star has a uniform surface brightness distribution.

##### Using the above equation and the light curve, estimate the radius of the planet Kepler-8b in units of Jupiter radius, $\mathrm{R_J}$. Include an estimation of the uncertainty on $R_p.$

In [ ]:
# Answer here

____

##### The calculated radius of the planet is 1.419 $\mathrm{M_J}$. Can you think of any reason why this is (slightly) different to the answer you calculated?

Answer here

____
If we view the transit side-on (from our observer frame on Earth), the equation for the duration time of the transit is:

$$\Delta t = \frac{P}{\pi} \sin^{-1}\left[\frac{R_*}{a}\right],$$

where $P$ is the period of the planet, $R_*$ is the radius of the star as before, and $a$ is the semimajor axis.

##### Using the above equation and the light curve, estimate the semimajor axis, $a$, of the planet Kepler-8b in units of AU.

In [ ]:
# Answer here

___
# Creating your own light curve

`.interact()` offers instantaneous interactive selection of the pixel mask, and instantaneous generation of the resulting light curve.  You can click on individual pixels and the aperture photometry seamlessly updates.  The mask can be defined with either individual clicking of pixels, or clicking and dragging a box over a rectangular set of pixels.  *De*selection of individual pixels works by re-clicking a pixel that you wish to take away from your mask.  Finally, you can save your mask and light curve as a FITS file by clicking on the `Save Lightcurve` button.

Let's first look at the target HL Tau, a young star that possesses a gapped circumstellar disk which has been [imaged by the Atacama Large Millimeter Array](http://www.almaobservatory.org/en/press-release/revolutionary-alma-image-reveals-planetary-genesis/).

In [ ]:
tpf = lk.search_targetpixelfile("HL Tau", author='K2', campaign=13, cadence='long').download()

The TPF of HL Tau contains a portion of a nearby source of comparable brightness.  The weakly overlapping point spread functions (PSFs) of these sources motivate some caution in aperture choice.  Let's interactively assign a custom aperture photometry pixel mask:

In [ ]:
tpf.interact()

You can move the large bottom left slider to change the location of the vertical red bar, which indicates which cadence is being shown in the TPF postage stamp image.  The slider beneath the TPF postage stamp image controls the screen stretch, which defaults to logarithmic scaling initialized to 1% and 95% lower and upper limits respectively.

You can move your cursor over individual data points to show hover-over tooltips indicating additional information about that datum. Currently the tooltips list the cadence, time, flux, and quality flags. The tools on the right hand side of the plots enable zooming and pixel selection.

We see that the starting mask (the *Kepler* pipeline mask, by default), shows huge jumps in flux between times 3000 and 3020.  These jagged artifacts disappear upon the selection of a larger aperture — large enough to encompass most of the point spread function of the star.  The end result shows a time series light curve of a young disk-bearing star.

##### Using the tools in the `.interact()` function, create your own light curve by deciding on the TPF mask.

Interaction modes:

- Clicking on a single pixel shows the time series light curve of that pixel alone.  
- `shift`-clicking on multiple pixels shows the light curve using that pixel mask. (*)
- `shift`-`ctrl`-clicking on an already selected pixel will *de*select that pixel. (May only work on Windows systems.)
- Clicking and dragging a box will make a rectangular aperture mask — individual pixels can be deselected from this mask by shift-clicking (box deselecting does not work).
- The screen stretch high and low limits can be changed independently by clicking and dragging each end, or simultaneously by clicking and dragging in the middle.
- The cadence slider updates the postage stamp image at the position of the vertical red bar in the light curve.
- Clicking on a position in the light curve automatically seeks to that cadence number.
- The left and right arrows can be clicked to increment the cadence number by one.
- (*) `shift`-clicking does not work on Jupyter Lab as of this writing.

##### Plot your light curve below, from the FITS file you have saved.
#####  Hint: remember the `read` function we used before.

In [ ]:
# Answer here

# Discover your own exoplanet signal

Using the tools we have discussed above, you can now recover your own exoplanet signals!

For the following TPF, determine the period of the transitting planet and plot the folded light curve to recover the transit signal.

The TPF corresponds to Kepler-10b, the first rocky planet that was discovered by Kepler!

In [ ]:
tpf = lk.search_targetpixelfile("Kepler-10", author="Kepler", quarter=3, cadence="long").download()

# Answer here

Whether a rocky explanet could be a suitable site for life depends directly on orbit. We therefore also would like to estimate how well the period is known. For this exercise you can make the estimate "by eye" through the interactive tools. 

What is your estimated uncertainty?

In [ ]:
# Answer here

# How old was the supernova ZTF19abqhobb when first observed?

The Zwicky Transient Facility discovered a new transient on Aug 19 2019 (JD=2458714.6629), and a novel program for _robotic_ follow-up of young supernovae was triggered. The SN was designated ZTF19abqhobb (a.k.a. "the Hobbit"). The autonomous sequence led to the a spectrum being taken on JD=2458714.7924631, roughly three hours after first detection. The spectrum shows a blue continuum, indicating a hot photosphere and a Type II Core Collapse supernovae where a massive giant phase star collapsed into a black hole. 

To determine the physical condiations at the explosion one would need to know the real age of the SN at the time of detection/spectroscopic observations. This is usually impossible to do accurately, but in this case we were lucky: the object happened to reside in one of the high cadence field observed by the TESS photometric satellite. Your task here is to use this data to measure the explosion time _and_ estimate the uncertainty of your measurement.

Note that the earliest ever observed spectra is thought to have been taken 8h and 11h after explosion, respectively. Could it be that this observation qualifies on this record list?



In [ ]:
# Tess data can also be retrieved through the lightkurve object.
# The SN was found at RA=261.411113 Dec=+59.446727 in Sector 15 of the TESS wide area campaign.
tessdata =lk.search_tesscut("261.411113 +59.446727", sector=15).download()

In [ ]:
# We define two masks, one for the centeral pixel where the SN resides and another for the remaining 24 pixels, 
# which we will use to estimate the background.
mask_supernova = np.zeros((5,5))
mask_supernova[2,2] = 1
mask_background = 1-mask_supernova

In [ ]:
lightcurve_supernova = tessdata.to_lightcurve(aperture_mask=mask_supernova)
lightcurve_background = tessdata.to_lightcurve(aperture_mask=mask_background)

In [ ]:
# Each TESS Sector last for some three weeks. We truncate the lightcurves to focus on the time right
# before and after the ZTF detection
lightcurve_supernova = lightcurve_supernova.truncate(before=1713,after=1720)
lightcurve_background = lightcurve_background.truncate(before=1713,after=1720)

In [ ]:
# Let us have a look at the lightcurves
%matplotlib inline
lightcurve_supernova.plot()
lightcurve_background.plot()

You will find the two lightcurves to look - _almost_ - identical and dominated by a long term trend together with oscillations. See if you find the time at which the supernova lightcurve starts to deviate above the background, i.e. when the SN exploded? How old was the SN when it was detected? 

In [ ]:
# Answer here

Finally, we need to estimate the uncertainty of this answer. Consider different sources of flux uncertainty (statistical, systematic, model) and how these would propagate to the final measurement.

In [ ]:
# Answer here

Note: This is real scientific question with no established answer yet.
Note 2: You might wanter why the TESS observation does not count as the detection. The problem has to do with data transfer, which is usually limited for a satellite orbiting far away and with strict reqiurements on power consumption. So the TESS telescope did observe the field of the SN first, but this data was not analyzed until far later, _after_ it had also been detected by ZTF.